In [ ]:
from summaryCentroids import kLLMmeans, kNLPmeans, get_embeddings, summarize_cluster
from experiment_utils import load_dataset, cluster_metrics, avg_closest_distance
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids

import numpy as np
import json, pickle

import warnings
warnings.filterwarnings("ignore")

In [ ]:
max_iter = 120

data_list = ['clinic','bank77','massive_I','massive_D','goemo']

for data in data_list:
    results_dict = {}
    
    with open("processed_data/data_" + data + ".pkl", "rb") as f:
        data_dict = pickle.load(f)
        
    labels = data_dict['labels']
    num_clusters = data_dict['num_clusters']   
    documents = data_dict['documents']
    text_features = data_dict['embeddings']
    #oracle_summaries = data_dict['summaries']
    prompt = data_dict['prompt']
    text_type = data_dict['text_type']

    with open("processed_data/data_sentences_" + data + ".pkl", "rb") as f:
        sentences_dict = pickle.load(f)
    
    oracle_cluster_assignments = labels
    
    for emb_type in ['distilbert', 'openai', 'e5-large', 'sbert']:
        
        results_dict[emb_type] = {}
        emb_data = text_features[emb_type]
        text_sentences = sentences_dict[emb_type]
        
        #calculate oracle embeddings
        oracle_clustered_embeddings = {i: [] for i in range(num_clusters)}
        for embedding, cluster in zip(emb_data, oracle_cluster_assignments):
            oracle_clustered_embeddings[cluster].append(embedding)
        oracle_centroids = [np.mean(oracle_clustered_embeddings[i], axis=0) if oracle_clustered_embeddings[i] else None for i in range(num_clusters)]
        #oracle_summary_embeddings = get_embeddings(oracle_summaries, emb_type = emb_type)
        oracle_summary_embeddings = oracle_centroids
        
        for seed in range(10):
            results_dict[emb_type][seed] = {}
    
            #kmeans
            kmeans = KMeans(n_clusters=num_clusters, max_iter=max_iter, random_state=seed)
            kmeans_assignments = kmeans.fit_predict(text_features[emb_type])
            kmeans_centroids = kmeans.cluster_centers_
            results = cluster_metrics(np.array(labels), kmeans_assignments, oracle_centroids, kmeans_centroids, oracle_summary_embeddings)
            data_results ={'assignments':kmeans_assignments,
                           'final_centroids':kmeans_centroids,
                           'results':results}
            
            results_dict[emb_type][seed]['kmeans'] = data_results
            
            print([data, emb_type, seed, 'kmeans', results])
    
            #kmedoids
            kmedoids = KMedoids(n_clusters=num_clusters, max_iter = max_iter, random_state=seed)
            kmedoids.fit(text_features[emb_type])
            kmedoids_assignments = kmedoids.labels_
            kmedoids_indices = kmedoids.medoid_indices_
            kmedoids_centroids = text_features[emb_type][kmedoids_indices]
            results = cluster_metrics(np.array(labels), kmedoids_assignments, oracle_centroids, kmedoids_centroids, oracle_summary_embeddings)
            data_results ={'assignments':kmedoids_assignments,
                           'final_centroids':kmedoids_centroids,
                           'results':results}
            
            results_dict[emb_type][seed]['kmedoids'] = data_results
            print([data, emb_type, seed, 'kmedoids', results])

            #gmm
            # Reduce to 50 principal components
            pca = PCA(n_components=50, random_state=seed)
            X_pca = pca.fit_transform(X_scaled)
            gmm = GaussianMixture(n_components=num_clusters, random_state=seed)
            gmm_assignments = gmm.fit_predict(X_pca)

            gmm_clustered_embeddings = {i: [] for i in range(num_clusters)}
            for embedding, cluster in zip(emb_data, gmm_assignments):
                gmm_clustered_embeddings[cluster].append(embedding)
            gmm_centroids = [np.mean(gmm_clustered_embeddings[i], axis=0) if gmm_clustered_embeddings[i] else None for i in range(num_clusters)]

            results = cluster_metrics(np.array(labels), gmm_assignments, gmm_centroids, oracle_centroids, oracle_centroids)
            data_results ={'assignments':gmm_assignments,
                           'results':results}
            results_dict[emb_type][seed]['gmm'] = data_results
            print([data, emb_type, seed, 'gmm', results])

            #spectral
            spectral = SpectralClustering(n_clusters=num_clusters, random_state=seed, affinity='nearest_neighbors')
            spectral_assignments = spectral.fit_predict(X_scaled)
            
            spectral_clustered_embeddings = {i: [] for i in range(num_clusters)}
            for embedding, cluster in zip(emb_data, spectral_assignments):
                spectral_clustered_embeddings[cluster].append(embedding)
            spectral_centroids = [np.mean(spectral_clustered_embeddings[i], axis=0) if spectral_clustered_embeddings[i] else None for i in range(num_clusters)]

            results = cluster_metrics(np.array(labels), spectral_assignments, spectral_centroids, oracle_centroids, oracle_centroids)
            data_results ={'assignments':spectral_assignments,
                           'results':results}
            results_dict[emb_type][seed]['spectral'] = data_results
            print([data, emb_type, seed, 'spectral', results])

            #Agglo
            agglo = AgglomerativeClustering(n_clusters=num_clusters)
            agglo_assignments = agglo.fit_predict(X_scaled)

            agglo_clustered_embeddings = {i: [] for i in range(num_clusters)}
            for embedding, cluster in zip(emb_data, agglo_assignments):
                agglo_clustered_embeddings[cluster].append(embedding)
            agglo_centroids = [np.mean(agglo_clustered_embeddings[i], axis=0) if agglo_clustered_embeddings[i] else None for i in range(num_clusters)]

            results = cluster_metrics(np.array(labels), agglo_assignments, agglo_centroids, oracle_centroids, oracle_centroids)
            data_results ={'assignments':agglo_assignments,
                           'results':results}
            results_dict[emb_type][seed]['agglomerative'] = data_results
            print([data, emb_type, seed, 'agglomerative', results])


            #k-LLMmeans
            for llm_type in ['gpt-3.5-turbo','gpt-4o','llama3.3-70b','deepseek-chat','claude-3-7-sonnet-20250219']:
                
                results_dict[emb_type][seed][llm_type] = {}

                #Uncomment to replicate only gpt-4o
                #if llm_type!='gpt-4o':
                #    continue
                
                for force_context_length in [0, 10]:
                    results_dict[emb_type][seed][llm_type][force_context_length] = {}
                                
                    for max_llm_iter in [1, 5]:
                        
                        assignments, final_summaries, final_summary_embeddings, final_centroids, summaries_evolution, centroids_evolution = kLLMmeans(documents,
                                                                 prompt = prompt, text_type = text_type,
                                                                 num_clusters = num_clusters, 
                                                                 force_context_length = force_context_length, max_llm_iter = max_llm_iter, 
                                                                 max_iter = max_iter, tol=1e-4, random_state = seed, 
                                                                 emb_type = emb_type,
                                                                 text_features = text_features[emb_type])
                        
                        results = cluster_metrics(np.array(labels), assignments,
                                                  oracle_centroids, final_centroids, 
                                                  oracle_summary_embeddings, final_summary_embeddings)
        
                        data_results ={'assignments':assignments,
                                       'final_summaries':final_summaries,
                                       'final_summary_embeddings':final_summary_embeddings,
                                       'final_centroids':final_centroids,
                                       'summaries_evolution':summaries_evolution,
                                       'centroids_evolution':centroids_evolution,
                                       'results':results}
                        
                        results_dict[emb_type][seed][llm_type][force_context_length][max_llm_iter] = data_results
                        
                        print([data, emb_type, llm_type, seed, force_context_length, max_llm_iter, results])
                        
                        # Save as pkl file
                        with open("results/sims_offline_results_" + emb_type + '_' + data + ".pkl", "wb") as f:
                            pickle.dump(results_dict, f)
                            
            #k-NLPmeans
            for nlp_type in ['lsa','centroid','textrank']:
                
                results_dict[emb_type][seed][nlp_type] = {}
               
                for top_k in [3, 5, 10, 15]:
                    results_dict[emb_type][seed][nlp_type][top_k] = {}
                                
                    for max_llm_iter in [1, 5]:

                        force_context_length = 0
                        assignments, final_summaries, final_summary_embeddings, final_centroids, summaries_evolution, centroids_evolution = kNLPmeans(documents,
                                                                 num_clusters = num_clusters, 
                                                                 force_context_length = force_context_length, max_llm_iter = max_llm_iter, 
                                                                 max_iter = max_iter, tol=1e-4, random_state = seed, 
                                                                 emb_type = emb_type,
                                                                 text_features = text_features[emb_type],
                                                                 top_k = top_k,
                                                                 text_sentences = text_sentences,
                                                                 nlp = nlp_type)
                        
                        results = cluster_metrics(np.array(labels), assignments,
                                                  oracle_centroids, final_centroids, 
                                                  oracle_summary_embeddings, final_summary_embeddings)
        
                        data_results ={'assignments':assignments,
                                       'final_summaries':final_summaries,
                                       'final_summary_embeddings':final_summary_embeddings,
                                       'final_centroids':final_centroids,
                                       'summaries_evolution':summaries_evolution,
                                       'centroids_evolution':centroids_evolution,
                                       'results':results}
                        
                        results_dict[emb_type][seed][nlp_type][top_k][max_llm_iter] = data_results
                        
                        print([data, emb_type, nlp_type, seed, top_k, max_llm_iter, results])
                        
                        # Save as pkl file
                        with open("results/sims_offline_results_" + emb_type + '_' + data + ".pkl", "wb") as f:
                            pickle.dump(results_dict, f)